# Part IV — Professional Django  
## 22. Architecture and Code Organization (How to Keep Django Clean at Scale)

As Django projects grow, most pain comes from **architecture drift**:

- views become huge (“fat views”)
- models accumulate unrelated logic (“fat models”)
- permissions are duplicated and inconsistent
- query logic is scattered and unoptimized
- forms/validation rules are repeated
- settings become a “big ball of mud”
- import cycles appear
- tests become slow and hard to maintain

This chapter gives you an industry-standard structure that stays maintainable from:
- 1 app / 10 endpoints
to
- 20 apps / 200 endpoints / multiple developers.

You will refactor your existing code (articles + orgs + tasks) into clean layers.

---

## 22.0 Learning Outcomes

By the end, you should be able to:

1. Define clean boundaries between:
   - views (HTTP boundary)
   - forms/serializers (input validation boundary)
   - selectors (read/query layer)
   - services (write/business logic layer)
   - domain rules (permissions/policies)
2. Avoid “fat views” and “fat models” while still using Django idiomatically.
3. Organize apps into maintainable modules:
   - `selectors.py`, `services.py`, `permissions.py`, `types.py`
4. Apply dependency inversion lightly (to keep code testable without overengineering).
5. Structure settings cleanly (base/dev/prod), and keep secrets out of code.
6. Create reusable internal packages and prevent circular imports.
7. Enforce architecture with conventions and tests.

---

## 22.1 Architecture Goals (What We’re Optimizing For)

A good Django architecture should:

- Make the “happy path” easy to implement quickly.
- Make bugs hard to introduce (centralized rules).
- Make changes safe (refactors don’t break random places).
- Make performance predictable (queries not scattered).
- Make testing easy (pure logic isolated from HTTP and DB where possible).

A common industry mantra:

- Keep your “web boundary” thin.
- Keep your “domain” explicit.
- Keep data access optimized and centralized.

---

## 22.2 Common Anti‑Patterns (And Why They Hurt)

### Anti-pattern A: Fat views
Symptoms:
- a single view function does parsing + validation + querying + permissions + saving
  + emailing + logging + branching
- you can’t reuse logic from admin, management commands, APIs

Result:
- copy/paste features
- inconsistent behavior
- hard-to-test code

### Anti-pattern B: Fat models
Symptoms:
- model methods contain:
  - permission checks
  - emailing
  - API calls
  - unrelated workflows

Result:
- hard to reason about side effects
- unexpected behavior from “just saving a model”
- circular import pressure

### Anti-pattern C: “Rules live in templates/UI”
Symptoms:
- you hide buttons for unauthorized users but don’t enforce in backend

Result:
- authorization bypass vulnerabilities

### Anti-pattern D: Query logic scattered everywhere
Symptoms:
- `.filter(...)` repeated in many views
- inconsistent `.select_related`/`.prefetch_related`
- N+1 regressions

Result:
- performance bugs and inconsistent results

---

## 22.3 The Practical Layered Pattern (Django-Friendly)

This is a simple, effective architecture used by many professional Django teams:

1. Views:
   - HTTP method handling
   - choose template/response
   - call services/selectors
2. Forms:
   - validate inputs (and normalize types)
3. Selectors (Read layer):
   - QuerySets and read-only query helpers
   - enforce scoping and prefetching patterns
4. Services (Write layer):
   - create/update workflows
   - transaction boundaries
   - domain events (email, logging, etc.)
5. Permissions/Policies:
   - “can user do X to object Y?”
   - centralized authorization logic
6. Models:
   - schema + minimal domain behavior tightly tied to the model itself
   - avoid external side effects in model methods

This is not “hexagonal architecture” with heavy abstractions. It’s lightweight and
Django-native.

---

## 22.4 Standard Module Layout Inside an App

For an app like `tasks`, a clean structure is:

```text
tasks/
  __init__.py
  admin.py
  apps.py
  models.py
  urls.py
  views.py                  # thin HTTP boundary
  forms.py                  # ModelForm(s)
  selectors.py              # read/query helpers
  services.py               # write/workflow helpers
  permissions.py            # authorization/policy rules
  types.py                  # dataclasses/TypedDicts (optional)
  tests/                    # optional: keep tests here (or global tests/)
    test_views.py
    test_services.py
```

Same idea applies to `articles` and `orgs`.

---

## 22.5 Refactor #1: Articles (Selectors + Services + Permissions)

Your current articles code likely has:
- list/detail queries inside views
- publish notification logic inside views
- ownership checks inside views

We will make it reusable and consistent.

### 22.5.1 Create `articles/selectors.py` (read layer)

```python
# articles/selectors.py
from __future__ import annotations

from django.db.models import Prefetch, QuerySet
from django.db.models import Q

from articles.models import Article, Tag


def article_qs_for_public() -> QuerySet[Article]:
    return (
        Article.objects.published()
        .select_related("author")
        .prefetch_related("tags")
    )


def article_qs_visible_to(user) -> QuerySet[Article]:
    return (
        Article.objects.visible_to(user)
        .select_related("author")
        .prefetch_related("tags")
    )


def filter_articles(
    *,
    qs: QuerySet[Article],
    q: str | None = None,
    tag: str | None = None,
) -> QuerySet[Article]:
    if tag:
        qs = qs.filter(tags__slug=tag)

    if q:
        q = q.strip()
        if q:
            qs = qs.filter(Q(title__icontains=q) | Q(body__icontains=q))

    return qs.distinct()
```

#### Why this is better than putting queries in views
- Every list/detail view uses the same prefetch/select_related strategy.
- Filtering rules are consistent.
- You can reuse selectors in:
  - admin custom pages
  - API endpoints later
  - management commands
  - tests

### 22.5.2 Create `articles/permissions.py` (policy layer)

```python
# articles/permissions.py
from __future__ import annotations

from articles.models import Article


def can_edit_article(*, user, article: Article) -> bool:
    if not user.is_authenticated:
        return False
    if user.is_staff:
        return True
    return article.author_id == user.id


def can_delete_article(*, user, article: Article) -> bool:
    # Example: same rule as edit; adjust as needed.
    return can_edit_article(user=user, article=article)
```

#### Why a permissions module matters
- Rules don’t drift across views.
- Tests become straightforward.
- When you add APIs later, you reuse the same policy.

### 22.5.3 Create `articles/services.py` (write/workflow layer)

This is where:
- transitions (draft → published)
- email sending
- audit logging (optional)
- transactions
should live.

```python
# articles/services.py
from __future__ import annotations

from dataclasses import dataclass

from django.db import transaction

from articles.models import Article
from articles.services_email import send_article_published_email


@dataclass(frozen=True)
class PublishResult:
    article: Article
    published_now: bool


@transaction.atomic
def update_article_from_form(*, form, actor) -> PublishResult:
    """
    Saves an Article form (create or edit), detects publish transition, and triggers
    side effects.

    - actor is the authenticated user performing the action
    - form is a ModelForm bound to an Article instance
    """
    article = form.instance
    old_status = article.status

    article = form.save(commit=False)

    if not getattr(article, "author_id", None):
        # Create flow typically sets author; enforce invariants here too.
        article.author = actor

    article.save()
    form.save_m2m()

    published_now = (
        old_status != Article.Status.PUBLISHED
        and article.status == Article.Status.PUBLISHED
    )

    if published_now:
        send_article_published_email(article=article)

    return PublishResult(article=article, published_now=published_now)
```

#### Why `transaction.atomic` here
If you later add:
- creating notifications rows
- logging rows
- updating related objects
you want all changes to succeed or fail together.

---

## 22.6 Refactor Views to Be Thin (Articles Example)

Now your view becomes orchestration:

```python
# articles/views.py
from __future__ import annotations

from django.contrib import messages
from django.core.exceptions import PermissionDenied
from django.core.paginator import EmptyPage, PageNotAnInteger, Paginator
from django.shortcuts import get_object_or_404, redirect, render
from django.views.decorators.http import require_GET, require_http_methods

from articles.forms import ArticleForm
from articles.forms_public import ArticleListFilterForm
from articles.permissions import can_edit_article
from articles.selectors import (
    article_qs_for_public,
    article_qs_visible_to,
    filter_articles,
)
from articles.services import update_article_from_form
from articles.models import Article


@require_GET
def article_list(request):
    form = ArticleListFilterForm(request.GET)
    form.is_valid()

    q = form.cleaned_data.get("q") if form.is_valid() else ""
    tag = form.cleaned_data.get("tag") if form.is_valid() else ""
    page_number = form.cleaned_data.get("page") if form.is_valid() else 1

    qs = article_qs_for_public()
    qs = filter_articles(qs=qs, q=q, tag=tag).order_by("-published_at", "-created_at")

    paginator = Paginator(qs, 10)
    try:
        page_obj = paginator.page(page_number or 1)
    except PageNotAnInteger:
        page_obj = paginator.page(1)
    except EmptyPage:
        page_obj = paginator.page(paginator.num_pages)

    return render(
        request,
        "articles/list.html",
        {
            "form": form,
            "articles": page_obj.object_list,
            "page_obj": page_obj,
            "q": q,
            "tag": tag,
        },
    )


@require_GET
def article_detail(request, slug: str):
    qs = article_qs_visible_to(request.user)
    article = get_object_or_404(qs, slug=slug)

    comments_qs = article.comments.filter(is_approved=True).order_by("created_at")

    return render(
        request,
        "articles/detail.html",
        {"article": article, "comments": comments_qs},
    )


@require_http_methods(["GET", "POST"])
def article_create(request):
    if not request.user.is_authenticated:
        return redirect("login")

    if request.method == "POST":
        form = ArticleForm(request.POST, request.FILES)
        if form.is_valid():
            result = update_article_from_form(form=form, actor=request.user)
            messages.success(request, "Article created.")
            return redirect("articles:detail", slug=result.article.slug)
    else:
        form = ArticleForm()

    return render(request, "articles/form.html", {"form": form, "mode": "create"})


@require_http_methods(["GET", "POST"])
def article_edit(request, slug: str):
    if not request.user.is_authenticated:
        return redirect("login")

    article = get_object_or_404(Article, slug=slug)

    if not can_edit_article(user=request.user, article=article):
        raise PermissionDenied

    if request.method == "POST":
        form = ArticleForm(request.POST, request.FILES, instance=article)
        if form.is_valid():
            result = update_article_from_form(form=form, actor=request.user)
            messages.success(request, "Article updated.")
            return redirect("articles:detail", slug=result.article.slug)
    else:
        form = ArticleForm(instance=article)

    return render(
        request,
        "articles/form.html",
        {"form": form, "mode": "edit", "article": article},
    )
```

### What improved
- views are readable and short
- publish transition/email is centralized (service)
- permissions are centralized (permissions module)
- query strategy and filtering are centralized (selectors)
- tests become easier (you can test services and permissions directly)

---

## 22.7 Refactor #2: Tasks (Centralize Scoping + Permissions + Query Building)

The tasks app is where architecture discipline really pays off because multi-user
apps tend to develop authorization leaks.

### 22.7.1 Create `tasks/permissions.py`

```python
# tasks/permissions.py
from __future__ import annotations

from orgs.models import Membership
from tasks.models import Task


def can_edit_task(*, membership: Membership, user, task: Task) -> bool:
    if membership.role == Membership.Role.ADMIN:
        return True
    return task.created_by_id == user.id or task.assigned_to_id == user.id


def can_delete_task(*, membership: Membership) -> bool:
    return membership.role == Membership.Role.ADMIN


def can_export_tasks(*, membership: Membership) -> bool:
    return membership.role == Membership.Role.ADMIN
```

### 22.7.2 Create `tasks/selectors.py`

```python
# tasks/selectors.py
from __future__ import annotations

from django.db.models import Q, QuerySet

from tasks.models import Task


def task_qs_for_org(*, organization) -> QuerySet[Task]:
    return Task.objects.filter(organization=organization).select_related(
        "assigned_to",
        "created_by",
        "updated_by",
    )


def filter_tasks(
    *,
    qs: QuerySet[Task],
    q: str | None = None,
    status: str | None = None,
    priority: int | None = None,
    assigned_to: str | None = None,
    actor=None,
) -> QuerySet[Task]:
    if q:
        q = q.strip()
        if q:
            qs = qs.filter(Q(title__icontains=q) | Q(description__icontains=q))

    if status:
        qs = qs.filter(status=status)

    if priority:
        qs = qs.filter(priority=priority)

    if assigned_to:
        if assigned_to == "me" and actor is not None:
            qs = qs.filter(assigned_to=actor)
        else:
            qs = qs.filter(assigned_to__username__iexact=assigned_to)

    return qs
```

### 22.7.3 Create `tasks/services.py` (write layer for create/update)

```python
# tasks/services.py
from __future__ import annotations

from dataclasses import dataclass

from django.db import transaction

from tasks.models import Task, TaskEvent


@dataclass(frozen=True)
class TaskSaveResult:
    task: Task
    created: bool


@transaction.atomic
def create_task_from_form(*, form, organization, actor) -> TaskSaveResult:
    task = form.save(commit=False)
    task.organization = organization
    task.created_by = actor
    task.updated_by = actor
    task.save()
    form.save_m2m()

    TaskEvent.objects.create(
        task=task,
        actor=actor,
        action="created",
        details={},
    )

    return TaskSaveResult(task=task, created=True)


@transaction.atomic
def update_task_from_form(*, form, actor) -> TaskSaveResult:
    task = form.instance
    old_status = task.status
    old_assignee_id = task.assigned_to_id

    task = form.save(commit=False)
    task.updated_by = actor
    task.save()
    form.save_m2m()

    if old_status != task.status:
        TaskEvent.objects.create(
            task=task,
            actor=actor,
            action="status_changed",
            details={"from": old_status, "to": task.status},
        )

    if old_assignee_id != task.assigned_to_id:
        TaskEvent.objects.create(
            task=task,
            actor=actor,
            action="assignee_changed",
            details={"from": old_assignee_id, "to": task.assigned_to_id},
        )

    return TaskSaveResult(task=task, created=False)
```

#### Why services for tasks matter
- audit/event logging is consistent
- updated_by is always set
- transactions ensure event and update happen together
- views stay small

---

## 22.8 Refactor Views (Tasks Example) to Use the Layers

Your task list view becomes:

```python
# tasks/views.py
from __future__ import annotations

import csv

from django.contrib.auth.decorators import login_required
from django.core.exceptions import PermissionDenied
from django.core.paginator import EmptyPage, PageNotAnInteger, Paginator
from django.http import HttpResponse
from django.shortcuts import get_object_or_404, redirect, render
from django.views.decorators.http import require_GET, require_http_methods

from orgs.services import get_membership, get_org_for_user_or_404
from tasks.forms import TaskForm
from tasks.forms_filters import TaskFilterForm
from tasks.permissions import (
    can_delete_task,
    can_edit_task,
    can_export_tasks,
)
from tasks.selectors import filter_tasks, task_qs_for_org
from tasks.services import create_task_from_form, update_task_from_form
from tasks.models import Task


@login_required
@require_GET
def task_list(request, org_slug: str):
    org = get_org_for_user_or_404(user=request.user, org_slug=org_slug)
    membership = get_membership(user=request.user, organization=org)

    form = TaskFilterForm(request.GET)
    form.is_valid()

    q = form.cleaned_data.get("q") if form.is_valid() else ""
    status = form.cleaned_data.get("status") if form.is_valid() else ""
    priority = form.cleaned_data.get("priority") if form.is_valid() else ""
    assigned_to = form.cleaned_data.get("assigned_to") if form.is_valid() else ""
    page_number = form.cleaned_data.get("page") if form.is_valid() else 1

    qs = task_qs_for_org(organization=org).order_by("-created_at")
    qs = filter_tasks(
        qs=qs,
        q=q,
        status=status or None,
        priority=int(priority) if priority else None,
        assigned_to=assigned_to or None,
        actor=request.user,
    )

    paginator = Paginator(qs, 20)
    try:
        page_obj = paginator.page(page_number or 1)
    except PageNotAnInteger:
        page_obj = paginator.page(1)
    except EmptyPage:
        page_obj = paginator.page(paginator.num_pages)

    return render(
        request,
        "tasks/list.html",
        {
            "organization": org,
            "membership": membership,
            "form": form,
            "tasks": page_obj.object_list,
            "page_obj": page_obj,
            "can_export": can_export_tasks(membership=membership),
        },
    )


@login_required
@require_http_methods(["GET", "POST"])
def task_create(request, org_slug: str):
    org = get_org_for_user_or_404(user=request.user, org_slug=org_slug)

    if request.method == "POST":
        form = TaskForm(request.POST)
        if form.is_valid():
            result = create_task_from_form(
                form=form,
                organization=org,
                actor=request.user,
            )
            return redirect(
                "tasks:detail",
                org_slug=org.slug,
                task_id=result.task.id,
            )
    else:
        form = TaskForm()

    return render(
        request,
        "tasks/form.html",
        {"organization": org, "form": form, "mode": "create"},
    )


@login_required
@require_http_methods(["GET", "POST"])
def task_edit(request, org_slug: str, task_id: int):
    org = get_org_for_user_or_404(user=request.user, org_slug=org_slug)
    membership = get_membership(user=request.user, organization=org)

    task = get_object_or_404(Task, id=task_id, organization=org)

    if not can_edit_task(membership=membership, user=request.user, task=task):
        raise PermissionDenied

    if request.method == "POST":
        form = TaskForm(request.POST, instance=task)
        if form.is_valid():
            result = update_task_from_form(form=form, actor=request.user)
            return redirect(
                "tasks:detail",
                org_slug=org.slug,
                task_id=result.task.id,
            )
    else:
        form = TaskForm(instance=task)

    return render(
        request,
        "tasks/form.html",
        {"organization": org, "form": form, "mode": "edit", "task": task},
    )


@login_required
@require_GET
def task_export_csv(request, org_slug: str):
    org = get_org_for_user_or_404(user=request.user, org_slug=org_slug)
    membership = get_membership(user=request.user, organization=org)

    if not can_export_tasks(membership=membership):
        raise PermissionDenied

    form = TaskFilterForm(request.GET)
    form.is_valid()

    qs = task_qs_for_org(organization=org).order_by("-created_at")
    qs = filter_tasks(
        qs=qs,
        q=form.cleaned_data.get("q") if form.is_valid() else "",
        status=form.cleaned_data.get("status") if form.is_valid() else "",
        priority=int(form.cleaned_data["priority"])
        if form.is_valid() and form.cleaned_data.get("priority")
        else None,
        assigned_to=form.cleaned_data.get("assigned_to") if form.is_valid() else "",
        actor=request.user,
    )

    response = HttpResponse(content_type="text/csv")
    response["Content-Disposition"] = (
        f'attachment; filename="{org.slug}-tasks.csv"'
    )

    writer = csv.writer(response)
    writer.writerow(["id", "title", "status", "priority", "assigned_to", "created_at"])

    for t in qs:
        writer.writerow(
            [
                t.id,
                t.title,
                t.status,
                t.priority,
                t.assigned_to.username if t.assigned_to else "",
                t.created_at.isoformat(),
            ]
        )

    return response
```

### The key win
Permissions + query + write workflows are centralized, so:
- adding DRF API later reuses the same services/selectors/policies
- preventing leaks is easier
- tests focus on policies and services

---

## 22.9 Where to Put Business Logic (A Practical Decision Table)

### Put logic in a model method when:
- it is purely about the model’s own state and invariants
- it does not trigger external side effects
- it is safe to call from many contexts

Example: a small helper property:

```python
class Article(models.Model):
    ...
    def is_public(self) -> bool:
        return self.status == Article.Status.PUBLISHED
```

### Put logic in a service when:
- it spans multiple models
- it must be transactional
- it sends email, logs events, triggers notifications
- it is a “workflow” (publish, assign, export, invite)

### Put logic in selectors when:
- it’s query composition
- it needs consistent select_related/prefetch_related
- it centralizes scoping and performance

### Put logic in forms when:
- it’s about input validation and normalization
- it should produce human-friendly error messages
- it protects from invalid/unsafe user input

---

## 22.10 Avoiding Circular Imports (A Django Scaling Pain Point)

Circular imports often happen when:
- models import services
- services import models
- apps import each other directly

### 22.10.1 Rules that prevent most circular imports
1. Models should not import services that import models.
2. Prefer importing inside functions in rare cases (local import).
3. Use string references in model relationships if needed:
   - `"orgs.Organization"`
4. Use `settings.AUTH_USER_MODEL` for user FKs.

### 22.10.2 Migrations and `apps.get_model`
In migrations, never import models directly. Use:

```python
Article = apps.get_model("articles", "Article")
```

You already practiced that.

---

## 22.11 Reusable Apps vs Project Apps (Industry Boundary)

- Some apps are clearly domain apps (articles, tasks, orgs).
- Some code is “project glue” (settings, middleware, context processors).

A common structure:

```text
config/                 # settings, urls, asgi/wsgi, middleware, logging config
core/                   # shared utilities, base models, common services
articles/ orgs/ tasks/  # domain apps
```

### 22.11.1 What belongs in `core/`
- small shared utilities (date parsing, id generation)
- shared middleware helpers
- base model mixins (timestamps, UUID PKs) when consistent across apps
- shared service helpers (send email wrapper, file validation helpers)
- shared exceptions (DomainError, PermissionError-like types)

But: don’t create `core` as a dumping ground. Keep it curated.

---

## 22.12 Settings Organization (Reinforce the Professional Pattern)

If you didn’t split settings yet, do it now (as in Security chapter):
- `config/settings/base.py`
- `config/settings/dev.py`
- `config/settings/prod.py`

Rules:
- base contains defaults safe for most environments
- dev contains dev-only things (debug toolbar, console email backend)
- prod contains strict security settings and env-based secrets

This prevents “production accidentally runs with dev settings.”

---

## 22.13 Dependency Inversion (Lightweight, Practical)

You do not need a heavy DI framework.

### 22.13.1 The simplest useful pattern: pass collaborators as parameters
Example: email sending.

Instead of hardcoding `msg.send()` inside the service, you can pass a sender for tests:

```python
def send_article_published_email(*, article, send_func=None) -> None:
    send_func = send_func or (lambda msg: msg.send())
    ...
    send_func(msg)
```

In tests, pass a fake `send_func` that captures messages.

This keeps code testable without big abstractions.

### 22.13.2 Another pattern: small interfaces (protocols) in typing-only code
Optional, advanced. Useful when integrating external services.

---

## 22.14 Testing the Architecture (What to Test at Each Layer)

- Permissions functions: unit tests (fast, no DB if possible)
- Selectors: integration tests (DB needed)
- Services: integration tests (DB + transactions + side effects mocked)
- Views: a smaller set of integration tests (ensure wiring works)

### Example: test `can_edit_task` without hitting DB
```python
from orgs.models import Membership
from tasks.permissions import can_edit_task


class Dummy:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)


def test_can_edit_task_member_rules():
    membership = Dummy(role=Membership.Role.MEMBER)
    user = Dummy(id=1, is_authenticated=True)
    task = Dummy(created_by_id=2, assigned_to_id=1)

    assert can_edit_task(membership=membership, user=user, task=task) is True
```

This kind of test is fast and robust.

---

# 22.15 LAB: Refactor Your Codebase to the Layered Structure

## Step 1 — Create modules
For each app:
- add `selectors.py`
- add `services.py`
- add `permissions.py`

## Step 2 — Move logic out of views
- move query building into selectors
- move publish/email transitions into services
- move ownership rules into permissions

## Step 3 — Ensure views are orchestration only
A good view should:
- validate input (forms)
- call service/selector
- return response

## Step 4 — Add tests for services and policies
- test service transitions (publish sends email once)
- test permissions for member/admin rules
- test selector scoping (non-member cannot see org tasks)

## Step 5 — Run linters/tests
```bash
python -m ruff check .
python -m black . --check
python manage.py test
```

---

## 22.16 Exercises (Do These Before Proceeding)

1. Create `orgs/selectors.py` and move membership queries there:
   - `is_member(user, org)` and `get_membership(user, org)`
2. Write a service `orgs/services_invites.py` (stub) that would send an invite email:
   - define function signature
   - write tests that it calls email send function (mocked)
3. Refactor CSV export filtering to reuse the same selector filter logic as the list
   view (no duplication).
4. Add a “domain exception” type in `core/exceptions.py` and use it in one service:
   - e.g., `DomainError("Cannot publish without title")`
5. Add a short “Architecture README” section documenting:
   - where to put queries
   - where to put workflows
   - where to put permissions

---

## 22.17 Chapter Summary (Keep This Mental Model)

- Thin views, explicit layers:
  - Forms validate inputs
  - Selectors query data (read layer)
  - Services perform workflows (write layer + transactions + side effects)
  - Permissions centralize authorization
- Keep models focused on schema and small domain helpers.
- Centralize scoping rules to prevent data leaks.
- Prevent circular imports by keeping dependencies one-way (views → services →
  models; not models → services).
- A consistent architecture makes your project easier to extend, test, and scale.

---

Next chapter: **Part V — 23. API Fundamentals**  
We’ll shift from server-rendered HTML to designing robust HTTP APIs (REST principles,
auth strategies, pagination standards, versioning, error formats) before we
implement APIs using Django REST Framework.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='21. logging_monitoring_scaling_foundations.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../5. Apis_with_Django/23. api_fundamentals.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
